## Weighted Erlang Case

In [19]:
import csv
import numpy as np
import math
from scipy.stats import binom, erlang, poisson
from scipy.optimize import minimize

In [20]:
def SCV_to_params(SCV):
    
    # weighted Erlang case
    if SCV <= 1:
        K = math.floor(1/SCV)
        p = ((K + 1) * SCV - math.sqrt((K + 1) * (1 - K * SCV))) / (SCV + 1)
        mu = K + (1 - p) * (K + 1)
    
        return K, p, mu
    
    # hyperexponential case
    else:
        p = 0.5 * (1 + np.sqrt((SCV - 1) / (SCV + 1)))
        mu = 1 # 1 / mean
        mu1 = 2 * p * mu
        mu2 = 2 * (1 - p) * mu
        
        return p, mu1, mu2

In [32]:
K, p, mu = SCV_to_params(0.99)
n = 3
omega = 0.5
Delta = 0.01
epsilon = 0.005
t_MAX = int(5/Delta)

In [37]:
with open(f'phase/SCV_1.00_minima_n3.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    old_minima = list(reader)

In [38]:
xi_matrix = [[[None for m in range(t_MAX)] for k in range(i+1)] for i in range(n)]
minima = [[[None for m in range(t_MAX)] for k in range(i+1)] for i in range(n)]

In [39]:
# K = 1
# p = 0.5
# t = 0.5
# mu = 1

# def f(k,t):
#     return poisson.sf(k-1, mu*t) * t - poisson.sf(k, mu*t) * k / mu


# def g_bar(k,z,t):
        
#     if 1 <= z <= K:
#         g_bar = 0
#         for l in range(k-1):
#             g_bar += (k - l - 1) * (sum([binom.pmf(m, l, 1-p) * f(l*K-z+1+m, t) for m in range(l+1)]) \
#                                     - sum([binom.pmf(m, l+1, 1-p) * f((l+1)*K-z+1+m, t) for m in range(l+2)]))
#     else:
#         g_bar = (k - 1) * (1 - np.exp(-mu * t)) / mu
# #         g_bar = (k - 1) * (t - f(1, t))
        
#         for l in range(1,k-1):
#             g_bar += (k - l - 1) * (sum([binom.pmf(m, l-1, 1-p) * f((l-1)*K+1+m, t) for m in range(l)]) \
#                                     - sum([binom.pmf(m, l, 1-p) * f(l*K+1+m, t) for m in range(l+1)]))
            
#     return g_bar

In [25]:
# for t in np.arange(0,10,0.1):
#     print(g_bar(2,1,t))

In [26]:
# mu = 1
# p = 0.5
# for t in np.arange(0,20,0.1):
# #     print(1 - t - np.exp(-t))
# #     print((3 - (3 + t)* np.exp(-t) ) / 2)
# #     print(t - (t / 2) * (poisson.sf(0, mu*t) + poisson.sf(1, mu*t)) + (1 / mu) * (poisson.sf(1, mu*t) + poisson.sf(2, mu*t)))

In [40]:
## STORAGE ###
N = int(50/Delta)

print("Step 1: Initialization")
B_sf = [0] * N
gamma = [[0] * N for z in range(K+1)]

# Poisson density and distribution
poisson_pmf = [[0] * N for z in range(n)]
poisson_cdf = [[0] * N for z in range(n)]

# Erlang distribution
erlang_cdf = [[0] * N for k in range((K+1)*n)]

# binomial densities
binom_pmf = [[binom.pmf(m, k, 1-p) for m in range(k+2)] for k in range(n)]

# P_k0
P_k0 = [[[0] * N for z in range(K+1)] for k in range(n)]

# f_bar and h_bar
f = [[0] * N for k in range((K+1)*n)]

f_bar = [[[0] * N for z in range(K+1)] for k in range(n+1)]
h_bar = [[0 for z in range(1,K+2)]] + \
    [[((k - 1) * (K + 1 - p) + 1 - z) / mu for z in range(1,K+1)] + [((k - 2) * (K + 1 - p) + 1) / mu] for k in range(2,n+1)]

# store as much as possible
print("Step 2: Computing subfunctions")
for t_ind in range(N):
    
    t = t_ind * Delta
    
    # Poisson
    for z in range(n):
        poisson_pmf[z][t_ind] = poisson.pmf(z, mu*t)
        poisson_cdf[z][t_ind] = poisson.cdf(z, mu*t)
    
    # gamma
    B_sf[t_ind] = poisson_cdf[K-1][t_ind] + (1 - p) * poisson_pmf[K][t_ind]
    
    for z in range(K):
        gamma[z][t_ind] = poisson_pmf[z][t_ind] / B_sf[t_ind]
    
    gamma[K][t_ind] = (1 - p) * poisson_pmf[K][t_ind] / B_sf[t_ind]
    
    # Erlang, f
    for k in range((K+1)*n):
        erlang_cdf[k][t_ind] = erlang.cdf(t, k, scale=1/mu)
        f[k][t_ind] = poisson.sf(k-1, mu*t) * t - poisson.sf(k, mu*t) * k / mu
    
    # P_k0, f_bar
    for k in range(n):
        for z in range(1,K+1):
            P_k0[k][z-1][t_ind] = sum([binom_pmf[k][m] * erlang_cdf[k*K-z+1+m][t_ind] for m in range(k+1)])
            f_bar[k][z-1][t_ind] = sum([binom_pmf[k][m] * f[k*K+1-z+m][t_ind] for m in range(k+1)])
        
        P_k0[k][K][t_ind] = sum([binom_pmf[k-1][m] * erlang_cdf[(k-1)*K+1+m][t_ind] for m in range(k)])
        f_bar[k][K][t_ind] = sum([binom_pmf[k-1][m] * f[(k-1)*K+1+m][t_ind] for m in range(k)])    

# f_circ, h_circ
print("Step 3: Computing f_circ and h_circ")
f_circ = [[[0] * N for m in range(t_MAX+1)] for k in range(n+1)]
h_circ = [[0] * (t_MAX+1) for k in range(n+1)]

for m in range(t_MAX+1):
    for k in range(n+1):
        h_circ[k][m] = sum([gamma[z-1][m] * h_bar[k-1][z-1] for z in range(1,K+2)])
        
        for t_ind in range(N):
            f_circ[k][m][t_ind] = sum([gamma[z-1][m] * f_bar[k][z-1][t_ind] for z in range(1, K+2)])


Step 1: Initialization
Step 2: Computing subfunctions
Step 3: Computing f_circ and h_circ


In [41]:
# 1. No client has been served before time t.
def P_up(u_ind, t_ind):
    """Computes P(N_t- = k | N_0 = k, B_0 = u)."""
    return B_sf[u_ind + t_ind] / B_sf[u_ind]

# 2. All clients have been served before time t.
def P_down(k, u_ind, t_ind):
    """Computes P(N_t- = 0 | N_0 = k, B_0 = u)."""
    return sum([gamma[z-1][u_ind] * P_k0[k][z-1][t_ind] for z in range(1,K+2)])

# 3. Some (but not all) clients have been served before time t.
def psi(v_ind, t_ind, k, l):
    """
    Computes P(t-v < Erl(k,mu) < t, Erl(k,mu) + Erl(l-k,mu) > t).
    """
    return sum([poisson_pmf[j][t_ind] * binom.sf(j-k, j, v_ind/t_ind) for j in range(k, l)])

def q(k, l, z, v, t_ind):
    """Computes P(N_t = l, \bar{B}_t < v | N_0 = k, Z_0 = z)."""
    
    v_ind = int(v/Delta)
    q = 0
    
    if z <= K:
        for m in range(k-l+2):
            I_klmz = (k - l + 1) * K - z + m + 1
            q += (p * psi(v_ind, t_ind, I_klmz, I_klmz+K) + (1 - p) * psi(v_ind, t_ind, I_klmz, I_klmz+K+1)) * binom_pmf[k-l+1][m]
    elif z == K + 1:
        for m in range(k-l+2):
            I_klm = (k - l) * K + m + 1
            q += (p * psi(v_ind, t_ind, I_klm, I_klm+K) + (1 - p) * psi(v_ind, t_ind, I_klm, I_klm+K+1)) * binom_pmf[k-l][m]
    
    return q

def compute_qs(k, l, t_ind):
    """Computes all the necessary q's only once."""
    
    qs = [[0] * (t_ind + 2) for z in range(K+1)]
    
    for z in range(1,K+2):
        qs[z-1][0] = q(k, l, z, 0, t_ind)
        
        for j in range(1, t_ind+1):
            qs[z-1][j] = q(k, l, z, (j - 0.5)*Delta, t_ind)
        
        qs[z-1][t_ind+1] = q(k, l, z, t_ind*Delta, t_ind)
    
    return qs

def q_bar(k, l, m, j, t_ind, qs):
    """Approximates P(N_{t*Delta} = l, B_{t*Delta} \in d(j*Delta) | N_0 = k, B_0 = m * Delta)."""
    return sum([gamma[z-1][m] * (qs[z-1][j+1] - qs[z-1][j]) for z in range(1, K+2)])

def cost_we(t_ind, i, k, m):
    """Computes (approximately) the cost when t is the next interarrival time."""
    
    cost = omega * f_circ[k][m][t_ind] + (1 - omega) * h_circ[k][m]
    cost += P_down(k, m, t_ind) * xi_we(i+1, 1, 0) + P_up(m, t_ind) * xi_we(i+1, k+1, m+t_ind)
    
    for l in range(2, k+1):
        
        qs = compute_qs(k, l, t_ind)
        
        for j in range(t_ind+1):
            
            cost_diff = q_bar(k, l, m, j, t_ind, qs) * xi_we(i+1, l, j)
            if cost_diff > 1e-10: ### TODO
                cost += cost_diff
    
    return cost

In [42]:
def xi_we(i, k, m):
    """Implements the weighted Erlang case."""
    
    # truncate time in service m
    if m > t_MAX:
        m = t_MAX
    
    if xi_matrix[i-1][k-1][m-1]: # retrieve stored value
        pass
    elif i == n: # initial condition
        xi_matrix[i-1][k-1][m-1] = (1 - omega) * h_circ[k][m]
    else:
        if m >= 3 and xi_matrix[i-1][k-1][m-2] and xi_matrix[i-1][k-1][m-3]:
            # fill all coming values with current cost & minimum
            if abs(xi_matrix[i-1][k-1][m-2] - xi_matrix[i-1][k-1][m-3]) < epsilon:
                xi_matrix[i-1][k-1][m-1:] = [xi_matrix[i-1][k-1][m-2]] * (t_MAX - (m - 1))
                minima[i-1][k-1][m-1:] = [minima[i-1][k-1][m-2]] * (t_MAX - (m - 1))

                print(i,k,m,"break")
                return xi_matrix[i-1][k-1][m-1]
        
        # initial guess
        if m > 0 and minima[i-1][k-1][m-2]:
            t_guess = minima[i-1][k-1][m-2]
            cost_guess = xi_matrix[i-1][k-1][m-2]
        else:
            t_guess = eval(old_minima[i-1][k-1])[m-1]
            cost_guess = cost_we(t_guess, i, k, m)

        t_new = t_guess
        
        # walk to the right
        while True:
            
            t_new += 1
            cost_new = cost_we(t_new, i, k, m)

            if cost_new < cost_guess:
                t_guess = t_new
                cost_guess = cost_new
            elif cost_new > cost_guess:
                break
        
        # walk to the left
        while True:

            t_new -= 1
            cost_new = cost_we(t_new, i, k, m)

            if cost_new < cost_guess:
                t_guess = t_new
                cost_guess = cost_new
            elif cost_new > cost_guess:
                break
        
        xi_matrix[i-1][k-1][m-1] = cost_guess
        minima[i-1][k-1][m-1] = t_guess
        
        print("end",i,k,m,t_guess,cost_guess)
    
    return xi_matrix[i-1][k-1][m-1]

In [216]:
# xi_we(1, 1, 0)

In [43]:
for i in np.arange(n,0,-1):
    for k in range(1,i+1):
        print("i =",i,"k =",k)
        for m in range(t_MAX):
            xi_we(i,k,m)

i = 3 k = 1
i = 3 k = 2
i = 3 k = 3
i = 2 k = 1
end 2 1 0 65 0.3230570565977996
end 2 1 1 65 0.3230381389170569
end 2 1 2 65 0.3230192543488392
2 1 3 break
i = 2 k = 2


IndexError: list index out of range

In [ ]:
# print("Cost:", xi_we(1, 1, 0))

In [ ]:
# with open('phase/SCV_0.99_omega_0.5_minima.csv','w', newline='') as myfile:
#     out = csv.writer(myfile)
#     out.writerows(minima)

# with open('phase/SCV_0.99_omega_0.5_cost.csv','w', newline='') as myfile:
#     out = csv.writer(myfile)
#     out.writerows(xi_matrix)

In [37]:
eval(old_minima[14-1][5-1])

[3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,
 3168,